# Instalamos librerias necesarias

In [ ]:
!pip install -q transformers torch sentence-transformers datasets scikit-learn
#!pip install -q faiss-cpu chromadb
!pip install -q google-generativeai
#!pip install -q pymupdf  # Para PDFs
!pip install -q accelerate

In [ ]:
import os
import re
import torch
#import fitz  # pymupdf
from google.colab import files
import warnings
import pandas as pd

## Habilitamos la api Gemini

In [ ]:
from google.colab import userdata
import os
google_api_key = userdata.get('GEMINI_API_KEY')

In [ ]:
import google.generativeai as genai

# Configurar API (necesitas tu API key)
genai.configure(api_key=google_api_key)

# Prompt egineering

## Modelos pagos

### gemini-1.5-flash

In [ ]:
# Inicializar modelo
model = genai.GenerativeModel('gemini-1.5-flash')

In [ ]:
prompt = """

Tengo 3 manzanas. Me como dos peras. ¿Cuántas manzanas me quedan?
"""

In [ ]:
response = model.generate_content(prompt)
result = response.text

In [ ]:
print(result)

Te quedan 3 manzanas.  Comer peras no afecta el número de manzanas que tienes.



### gemini-2.0-flash-exp

In [ ]:
# Configurar parámetros del modelo
temperatura = 0.7  # 0.0 = muy determinista, 1.0 = muy creativo
top_p = 0.8       # 0.1 = conservador, 1.0 = diverso
max_tokens = 500  # máximo de palabras en la respuesta

# Configuración de generación
generation_config = genai.types.GenerationConfig(
    temperature=temperatura,
    top_p=top_p,
    max_output_tokens=max_tokens
)


# Inicializar modelo con configuración
model = genai.GenerativeModel(
    'gemini-2.0-flash-exp',
    generation_config=generation_config
)

In [ ]:
prompt = """

Tengo 3 manzanas. Me como dos peras. ¿Cuántas manzanas me quedan?
"""

response = model.generate_content(prompt)
result = response.text

In [ ]:
print(result)

Te quedan 3 manzanas. El hecho de que te comas dos peras no afecta la cantidad de manzanas que tienes.



## Probando modelos OpenSource

### Llama-3.2-1B

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
# Cargar modelo y tokenizer
model_name = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [ ]:
# Configurar pad_token para evitar warnings
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
prompt = """
Tengo 3 manzanas. Me como dos peras. ¿Cuántas manzanas me quedan?
"""

# Generar respuesta (estructura similar a Gemini)
inputs = tokenizer(prompt, return_tensors="pt", padding=True)
outputs = model.generate(
    inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=50,
    pad_token_id=tokenizer.eos_token_id
)

In [ ]:
# Decodificar resultado completo
result = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(result)

=== RESULTADO COMPLETO ===

Tengo 3 manzanas. Me como dos peras. ¿Cuántas manzanas me quedan?
> **a) 3 manzanas**. 3 manzanas.
> **b) 2 manzanas**. 2 manzanas.
> **c) 1 manzana**. 1 manzana


In [ ]:
# También mostrar solo la respuesta nueva
response_text = result[len(prompt):].strip()

print(response_text)


=== SOLO LA RESPUESTA ===
> **a) 3 manzanas**. 3 manzanas.
> **b) 2 manzanas**. 2 manzanas.
> **c) 1 manzana**. 1 manzana


### Meta-Llama-3-8B"

In [ ]:
# Cargar modelo y tokenizer
model_name = "meta-llama/Meta-Llama-3-8B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [ ]:
# Configurar pad_token para evitar warnings
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
prompt = """
Tengo 3 manzanas. Me como dos peras. ¿Cuántas manzanas me quedan?
"""

# Generar respuesta
inputs = tokenizer(prompt, return_tensors="pt", padding=True)
outputs = model.generate(
    inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=50,
    pad_token_id=tokenizer.eos_token_id,
    do_sample=True,  # Esto ayuda a generar texto nuevo
    temperature=0.7  # Un poco de aleatoriedad
)

In [ ]:
# También mostrar solo la respuesta nueva

result = tokenizer.decode(outputs[0], skip_special_tokens=True)
response_text = result[len(prompt):].strip()

print(response_text)

¿Cuántas manzanas comí?
¿Cuántas manzanas tenia al principio?
¿Cuántas manzanas me quedan al final?

# Tengo 3 manzanas. Me como dos peras.


## Hands on!

### Cargamos datos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
tweets_path = '/content/drive/MyDrive/Clases/NLP para ciencias sociales/99. Datos/tweets_sentiment_ejercicios_simple.csv'

df=pd.read_csv(tweets_path)

df.head()

,texto,sentiment_label,sentiment_transformer
0,Después de escribir durante algún tiempo en “L...,NEU,NEU
1,Transportaban el ÑÑ en aviones militares cuand...,NEG,NEU
2,"Hace poco Uribe propuso una amnistía general, ...",NEG,NEU
3,Palabras ante diputados y concejales del Centr...,NEU,NEU
4,Aquí está la tasa de homicidios en el Cauca en...,NEG,NEU


## Prompting

In [ ]:
# Configurar parámetros del modelo
temperatura = 0.7  # 0.0 = muy determinista, 1.0 = muy creativo
top_p = 0.8       # 0.1 = conservador, 1.0 = diverso
max_tokens = 50  # máximo de palabras en la respuesta

# Configuración de generación
generation_config = genai.types.GenerationConfig(
    temperature=temperatura,
    top_p=top_p,
    max_output_tokens=max_tokens
)


# Inicializar modelo con configuración
model = genai.GenerativeModel(
    'gemini-2.0-flash-exp',
    generation_config=generation_config
)

In [ ]:
# Crear prompt para análisis de sentimientos
def crear_prompt_sentimiento(texto_tweet):
    prompt = f"""
Analiza el sentimiento del siguiente tweet político colombiano:

TWEET: "{texto_tweet}"

Clasifica el sentimiento como:
- POSITIVO: optimista, de apoyo, constructivo
- NEGATIVO: crítico, pesimista, destructivo
- NEUTRAL: informativo, objetivo, sin carga emocional clara

Responde SOLO con una palabra: POSITIVO, NEGATIVO o NEUTRAL
"""
    return prompt

In [ ]:
# 3. Probar con los primeros 5 tweets

import time

resultados = []
predicciones_gemini = []

for i in range(40):
    tweet = df.iloc[i]
    texto = tweet['texto']
    sentiment_pysentimiento = tweet['sentiment_label']
    sentiment_transformer = tweet['sentiment_transformer']

    # Crear prompt
    prompt = crear_prompt_sentimiento(texto)

    # Generar respuesta con Gemini
    response = model.generate_content(prompt)
    prediccion_gemini = response.text.strip()

    # Agregar a la lista para el DataFrame final
    predicciones_gemini.append(prediccion_gemini)

    # Guardar resultado
    resultado = {
        'texto': texto[:100] + "..." if len(texto) > 100 else texto,
        'pysentimiento': sentiment_pysentimiento,
        'transformer': sentiment_transformer,
        'gemini': prediccion_gemini
    }
    resultados.append(resultado)

    print(f"Tweet {i+1}/40 procesado")

    # Pausa cada 5 solicitudes
    if (i + 1) % 5 == 0:
        print(f"Pausa de 60 segundos...")
        time.sleep(60)

Tweet 1/40 procesado
Tweet 2/40 procesado
Tweet 3/40 procesado
Tweet 4/40 procesado
Tweet 5/40 procesado
Pausa de 60 segundos...
Tweet 6/40 procesado
Tweet 7/40 procesado
Tweet 8/40 procesado
Tweet 9/40 procesado
Tweet 10/40 procesado
Pausa de 60 segundos...
Tweet 11/40 procesado
Tweet 12/40 procesado
Tweet 13/40 procesado
Tweet 14/40 procesado
Tweet 15/40 procesado
Pausa de 60 segundos...
Tweet 16/40 procesado
Tweet 17/40 procesado
Tweet 18/40 procesado
Tweet 19/40 procesado
Tweet 20/40 procesado
Pausa de 60 segundos...
Tweet 21/40 procesado
Tweet 22/40 procesado
Tweet 23/40 procesado
Tweet 24/40 procesado
Tweet 25/40 procesado
Pausa de 60 segundos...
Tweet 26/40 procesado
Tweet 27/40 procesado
Tweet 28/40 procesado
Tweet 29/40 procesado
Tweet 30/40 procesado
Pausa de 60 segundos...
Tweet 31/40 procesado
Tweet 32/40 procesado
Tweet 33/40 procesado
Tweet 34/40 procesado
Tweet 35/40 procesado
Pausa de 60 segundos...
Tweet 36/40 procesado
Tweet 37/40 procesado
Tweet 38/40 procesado
Tweet

In [ ]:
# Crear DataFrame final con nueva columna
df_final = df.copy()
df_final['prediccion_gemini'] = predicciones_gemini

print("Procesamiento completado!")
print(df_final[['texto', 'sentiment_label', 'sentiment_transformer', 'prediccion_gemini']].head())

Procesamiento completado!
                                               texto sentiment_label  \
0  Después de escribir durante algún tiempo en “L...             NEU   
1  Transportaban el ÑÑ en aviones militares cuand...             NEG   
2  Hace poco Uribe propuso una amnistía general, ...             NEG   
3  Palabras ante diputados y concejales del Centr...             NEU   
4  Aquí está la tasa de homicidios en el Cauca en...             NEG   

  sentiment_transformer prediccion_gemini  
0                   NEU           NEUTRAL  
1                   NEU          NEGATIVO  
2                   NEU          NEGATIVO  
3                   NEU           NEUTRAL  
4                   NEU          NEGATIVO  


In [ ]:
df_final.to_excel("twiter_sentiment_pro.xlsx")

In [ ]:
df_final.to_csv("twiter_sentiment_pro.csv")

#Fine tunning

In [ ]:
#import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.metrics import accuracy_score
import torch

In [ ]:
tweets_path = '/content/drive/MyDrive/Clases/NLP para ciencias sociales/99. Datos/twitter_sentiment_FT.xlsx'
df_FT=pd.read_excel(tweets_path)

In [ ]:
# Dividir en entrenamiento (30) y prueba (10)
train_df = df_final.head(30).copy()
test_df = df_final.tail(10).copy()

print(f"Datos de entrenamiento: {len(train_df)} tweets")
print(f"Datos de prueba: {len(test_df)} tweets")

Datos de entrenamiento: 30 tweets
Datos de prueba: 10 tweets


In [ ]:
# Usar sentiment_label como target (POS, NEG, NEU)
# Convertir a números
label_map = {'POS': 2, 'NEG': 0, 'NEU': 1}
reverse_label_map = {2: 'POS', 0: 'NEG', 1: 'NEU'}

train_df['labels'] = train_df['sentiment_label'].map(label_map)
test_df['labels'] = test_df['sentiment_label'].map(label_map)

print("Distribución de entrenamiento:")
print(train_df['sentiment_label'].value_counts())

Distribución de entrenamiento:
sentiment_label
NEG    14
NEU     9
POS     7
Name: count, dtype: int64


## Cargamos modelo pre-entrenado pequeño

In [ ]:
model_name = "distilbert-base-uncased"  # Modelo pequeño y rápido
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Tokenizar los datos
def tokenize_function(examples):
    return tokenizer(examples['texto'], truncation=True, padding=True, max_length=128)

# Crear datasets
train_dataset = Dataset.from_pandas(train_df[['texto', 'labels']])
test_dataset = Dataset.from_pandas(test_df[['texto', 'labels']])

# Tokenizar
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/30 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
# 4. Configurar entrenamiento (con explicaciones)
training_args = TrainingArguments(
    output_dir='/content/results',       # 📁 Carpeta donde guardar checkpoints y logs
    num_train_epochs=3,                  # 🔄 Cuántas veces pasar por todos los datos (3 vueltas completas)
    per_device_train_batch_size=4,       # 📦 Cuántos tweets procesar juntos en entrenamiento (4 a la vez)
    per_device_eval_batch_size=4,        # 📦 Cuántos tweets procesar juntos en evaluación (4 a la vez)
    warmup_steps=10,                     # 🔥 Pasos para "calentar" el learning rate (empezar despacio)
    weight_decay=0.01,                   # ⚖️ Regularización para evitar overfitting (0.01 = poquito)
    logging_dir='/content/logs',         # 📊 Carpeta para guardar métricas y gráficos
    save_steps=50,                       # 💾 Cada cuántos pasos guardar el modelo (cada 50 pasos)
    eval_strategy="steps",               # 📈 Cuándo evaluar: "steps"=cada X pasos, "epoch"=cada época
    eval_steps=50                        # 📈 Cada cuántos pasos evaluar (cada 50 pasos)
)

In [ ]:
# Función para calcular métricas (también con explicación)
def compute_metrics(eval_pred):
    """
    Función que calcula métricas de evaluación

    eval_pred: contiene las predicciones del modelo y las etiquetas reales
    - predictions: probabilidades para cada clase [POS, NEU, NEG]
    - labels: etiquetas reales (0, 1, 2)
    """
    predictions, labels = eval_pred

    # Convertir probabilidades a predicciones finales
    # argmax = tomar la clase con mayor probabilidad
    predictions = predictions.argmax(axis=-1)

    # Calcular accuracy = % de predicciones correctas
    accuracy = accuracy_score(labels, predictions)

    return {'accuracy': accuracy}

In [ ]:
# Crear trainer
# Es como un 'profesor' que entrena el modelo. Toma el modelo base (DistilBERT),
#le enseña con nuestros 30 tweets etiquetados") y cada tanto lo evalúa con los 10 tweets de prueba.
# Ajusta los pesos del modelo para mejorar predicciones, guarda el progreso y métricas.

os.environ["WANDB_DISABLED"] = "true"

# Configurar entrenamiento SIN wandb
training_args = TrainingArguments(
    output_dir='/content/results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=10,
    weight_decay=0.01,
    logging_dir='/content/logs',
    save_steps=50,
    eval_strategy="steps",
    eval_steps=50,
    report_to=[]  #Esto deshabilita todos los reportes externos
)

In [ ]:
# Crear trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)


In [ ]:
# evaluamos

# Hacer predicciones en los 10 tweets de prueba
predictions = trainer.predict(test_dataset)
predicted_labels = predictions.predictions.argmax(axis=-1)

# Convertir números de vuelta a etiquetas
reverse_label_map = {2: 'POS', 0: 'NEG', 1: 'NEU'}
predicted_sentiment = [reverse_label_map[pred] for pred in predicted_labels]

### Mostramos resultados

In [ ]:
for i in range(len(test_df)):
    tweet_real = test_df.iloc[i]
    texto_corto = tweet_real['texto'][:60] + "..." if len(tweet_real['texto']) > 60 else tweet_real['texto']

    print(f"Tweet {i+1}:")
    print(f"Texto: {texto_corto}")
    print(f"Real: {tweet_real['sentiment_label']}")
    print(f"Predicción: {predicted_sentiment[i]}")

    # Verificar si acertó
    if tweet_real['sentiment_label'] == predicted_sentiment[i]:
        print("✅ Correcto")
    else:
        print("❌ Incorrecto")
    print("-" * 50)

# CALCULAR ACCURACY
test_labels = test_df['sentiment_label'].tolist()
aciertos = sum(1 for real, pred in zip(test_labels, predicted_sentiment) if real == pred)
accuracy = aciertos / len(test_labels)

print(f"\n📊 RESULTADOS FINALES:")
print(f"Tweets de prueba: {len(test_labels)}")
print(f"Aciertos: {aciertos}")
print(f"Accuracy: {accuracy:.2f} ({accuracy*100:.1f}%)")

# GUARDAR MODELO
#trainer.save_model('/content/modelo_finetuned_sentimientos')
#print(f"\n✅ Modelo guardado en '/content/modelo_finetuned_sentimientos'")

Tweet 1:
Texto: #DebatesCaracolRadio
Nos vemos a las 6:00 PM por Caracol Rad...
Real: NEU
Predicción: NEG
❌ Incorrecto
--------------------------------------------------
Tweet 2:
Texto: Un saludo muy especial a La Estrella, Antioquia, por sus más...
Real: POS
Predicción: NEG
❌ Incorrecto
--------------------------------------------------
Tweet 3:
Texto: El presidiario ha desatado la violencia desde el Estado que ...
Real: NEG
Predicción: NEG
✅ Correcto
--------------------------------------------------
Tweet 4:
Texto: Por primera vez preguntan en una encuesta por el Pacto Histó...
Real: NEU
Predicción: NEG
❌ Incorrecto
--------------------------------------------------
Tweet 5:
Texto: Firmado contrato de concesión de Puerto Antioquia; anuncian ...
Real: NEU
Predicción: NEG
❌ Incorrecto
--------------------------------------------------
Tweet 6:
Texto: #Petro90Días 
—10 grupos armados están en cese unilateral al...
Real: NEU
Predicción: NEG
❌ Incorrecto
---------------------------------

# RAG